In [1]:
#importamos módulos básicos
import os
# PIL de pillow es un buen módulo para procesar imágenes
import PIL
import PIL.Image
import numpy as np

In [2]:
# instalamos módulos para procesar rostros
!pip install -U insightface
!pip install onnxruntime
from insightface.app import FaceAnalysis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 56.6 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp311-cp311-linux_x86_64.whl size=1060438 sha256=7bcc4595d78ba59d25163d7db5f1097428d0606122c17a0fd4971e37f473d692
  Stored in directory: /root/.cache/pip/wheels/27/d8/22/f52d858d16cd06e7b2e6aad34a1777dcfaf000be833bbf8146
Successfully built insightface
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.7 MB/s eta 0:00:00


In [3]:
# descargar un pequeño dataset de prueba
!wget "https://www.dropbox.com/scl/fi/b9y03enpi01zfb36ckyir/personas.zip?rlkey=19wypqmaps5b5n28bz3gi7xg7&st=s6p3564g&dl=0" -O personas.zip

--2025-07-24 00:55:58--  https://www.dropbox.com/scl/fi/b9y03enpi01zfb36ckyir/personas.zip?rlkey=19wypqmaps5b5n28bz3gi7xg7&st=s6p3564g&dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc74e0cb917352a000ea9a0f683b.dl.dropboxusercontent.com/cd/0/inline/CuGGJIfpIGO_T-Yz-ELXW35ThGpBCkcMr7jxD5obG40l6axLioGryOm6LqcF9In1FoR_lzs0dpn65Ewi7w7_0-Q44ZlMAF49W0Ga4QumSJEXGsLvlA8S8_Y_gghqwWM4hTWwqx1O7zWf7f7F1nVif1ph/file# [following]
--2025-07-24 00:55:59--  https://uc74e0cb917352a000ea9a0f683b.dl.dropboxusercontent.com/cd/0/inline/CuGGJIfpIGO_T-Yz-ELXW35ThGpBCkcMr7jxD5obG40l6axLioGryOm6LqcF9In1FoR_lzs0dpn65Ewi7w7_0-Q44ZlMAF49W0Ga4QumSJEXGsLvlA8S8_Y_gghqwWM4hTWwqx1O7zWf7f7F1nVif1ph/file
Resolving uc74e0cb917352a000ea9a0f683b.dl.dropboxusercontent.com (uc74e0cb917352a000ea9a0f683b.dl.dropboxusercontent.com)... 162

In [4]:
# descomprimimos el dataset con 7 personalidades
!unzip personas.zip

Archive:  personas.zip
   creating: personas/
   creating: personas/barack/
  inflating: personas/barack/face_2.jpeg  
  inflating: personas/barack/barack.jpeg  
  inflating: personas/barack/barack_2.jpeg  
   creating: personas/dicaprio/
  inflating: personas/dicaprio/dicaprio_2.png  
  inflating: personas/dicaprio/dicaprio.jpg  
   creating: personas/pancho/
  inflating: personas/pancho/pancho.jpeg  
  inflating: personas/pancho/pancho2.jpeg  
   creating: personas/thalia/
  inflating: personas/thalia/thalia2.jpg  
  inflating: personas/thalia/thalia1.jpg  
   creating: personas/paulina/
  inflating: personas/paulina/paulina2.png  
  inflating: personas/paulina/paulina1.jpeg  
  inflating: personas/faces.txt      
   creating: personas/mario/
  inflating: personas/mario/mario1.png  
  inflating: personas/mario/mario2.png  
   creating: personas/michelle/
  inflating: personas/michelle/michelle_obama_2.jpg  
  inflating: personas/michelle/michelle_obama_1.jpeg  


In [21]:
datapath = '/content/personas'
ffaces = os.path.join(datapath, 'faces.txt')
fdata = []
with open(ffaces) as f:
  for line in f :
    iname, icl = line.split()
    iname = os.path.join(datapath, iname)
    fdata.append({'path': iname, 'name': icl})
print(fdata)

[{'path': '/content/personas/paulina/paulina2.png', 'name': 'paulina'}, {'path': '/content/personas/paulina/paulina1.jpeg', 'name': 'paulina'}, {'path': '/content/personas/dicaprio/dicaprio_2.png', 'name': 'dicaprio'}, {'path': '/content/personas/dicaprio/dicaprio.jpg', 'name': 'dicaprio'}, {'path': '/content/personas/barack/barack.jpeg', 'name': 'barack'}, {'path': '/content/personas/barack/barack_2.jpeg', 'name': 'barack'}, {'path': '/content/personas/barack/face_2.jpeg', 'name': 'barack'}, {'path': '/content/personas/pancho/pancho.jpeg', 'name': 'pancho'}, {'path': '/content/personas/pancho/pancho2.jpeg', 'name': 'pancho'}, {'path': '/content/personas/thalia/thalia1.jpg', 'name': 'thalia'}, {'path': '/content/personas/thalia/thalia2.jpg', 'name': 'thalia'}, {'path': '/content/personas/mario/mario1.png', 'name': 'mario'}, {'path': '/content/personas/mario/mario2.png', 'name': 'mario'}, {'path': '/content/personas/michelle/michelle_obama_1.jpeg', 'name': 'michelle'}, {'path': '/conten

In [15]:
# inicializamos el modelo de rostros
app = FaceAnalysis(name='buffalo_l', providers=['CPUExecutionProvider'])  # Use 'CUDAExecutionProvider' for GPU
app.prepare(ctx_id=0)  # ctx_id=-1 for CPU, 0 for GPU

def get_face_embedding(image_path):
    """Extract face embedding from an image"""
    img = PIL.Image.open(image_path)
    if img is None:
        raise ValueError(f"Could not read image: {image_path}")
    img = np.array(img)
    if img.shape[2] == 4 :
      img = img[:,:,:-1]
    faces = app.get(img)
    if len(faces) < 1:
        raise ValueError("No faces detected in the image")
    if len(faces) > 1:
        print("Warning: Multiple faces detected. Using first detected face")
    # retorna el primer rostro junto con su embedding
    return faces[0].embedding

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [22]:
# recorremos la lista de rostros
embs = []
names = []
compute = True
for person in fdata:
    emb = get_face_embedding(iname)
    person['emb'] = emb

{'path': '/content/personas/paulina/paulina2.png', 'name': 'paulina', 'emb': array([ 7.05243424e-02,  3.57143283e-02,  7.93796718e-01, -2.73229420e-01,
        2.78662980e-01,  1.30508459e+00,  6.63555026e-01,  8.50216746e-01,
        1.34386718e+00, -1.38272762e+00,  2.20376521e-01,  7.85771012e-01,
        2.80128628e-01,  1.02538013e+00,  2.37646192e-01, -2.82160997e-01,
       -7.98470378e-01, -6.83651090e-01, -1.22037971e+00,  3.76102269e-01,
       -6.07256532e-01, -4.76888083e-02, -2.40821406e-01, -6.88033402e-01,
        1.78495777e+00,  1.56535208e+00,  1.24066460e+00,  1.08458020e-01,
       -4.75189239e-02, -1.35807693e+00,  3.77980731e-02, -3.31163220e-02,
       -1.52768850e+00, -8.25949460e-02, -4.05757964e-01,  1.04392469e+00,
        1.71967697e+00, -9.31400001e-01,  2.16844752e-01, -1.48251390e+00,
        1.07953310e+00,  2.31733227e+00, -3.33844066e-01,  1.49236560e-01,
       -1.27546966e+00,  3.61777186e-01,  6.75605655e-01, -1.04720306e+00,
       -3.25421154e-01,

In [ ]:
# formar una matriz de embeddings
n = len(fdata)
dim = len(fdata[0]['emb'])
embs = np.array((n, dim), dtype = np.float32)
for person in fdata:


In [17]:
print(embs.shape)
print(embs[2,:])

(15, 512)
[-8.60199749e-01 -3.25832903e-01 -5.88276088e-01 -3.00620079e-01
 -8.68814945e-01  1.08419919e+00 -1.78388691e+00  8.59658062e-01
  2.10448369e-01 -1.12748790e+00  1.81425714e+00  1.48629332e+00
  7.14112282e-01  9.10298049e-01 -1.64289880e+00 -1.86040089e-01
 -1.01427805e+00 -5.27941585e-02 -1.02015615e+00  1.68734074e+00
 -8.31181467e-01  7.06269801e-01  1.77542400e+00  8.04062665e-01
 -2.25114688e-01  1.16627622e+00  1.16601133e+00 -9.17855263e-01
  5.21242976e-01 -4.10577714e-01  4.25215483e-01 -1.12431669e+00
 -1.41559947e+00 -6.14801407e-01  6.19383931e-01  1.58839738e+00
  2.08503246e+00 -7.65935630e-02 -1.48317367e-02 -8.30320477e-01
 -1.42843449e+00 -1.27630651e+00  5.09456754e-01  1.19214725e+00
 -1.42661095e+00 -1.20847917e+00 -8.57758641e-01  4.50519145e-01
 -1.76722556e-02  3.51974159e-01 -3.73316914e-01  3.11537683e-01
  3.64473760e-02  1.77025378e+00 -4.81345594e-01 -4.38780367e-01
 -2.07912460e-01  8.78964365e-01 -1.99835092e-01  1.38903046e+00
 -9.30834711e-0

In [18]:
#np.save("face_embs.npy", embs)
# embs = np.load("face_embs.npy")
#similarity search
embs_norm = embs / np.linalg.norm(embs, ord = 2, axis = 1,  keepdims = True)
sim = embs_norm @ np.transpose(embs_norm)
print(sim)


[[ 1.0000000e+00  6.2091148e-01  1.4419974e-01  1.1881724e-01
  -4.3928705e-02 -4.2916708e-02 -4.6245165e-02  1.0602462e-01
   6.0486905e-02 -9.2999907e-03  1.1417773e-01  4.8244365e-02
   1.9483801e-02  1.1305122e-01  1.2899187e-01]
 [ 6.2091148e-01  1.0000000e+00  1.5393433e-01  9.2735447e-02
  -8.4229112e-02 -4.6397049e-02 -1.9339167e-02  1.1954491e-01
   7.1693614e-02  2.9089380e-02  1.3622121e-02  2.8045475e-04
  -1.1759914e-02  3.0222097e-03 -4.5506060e-03]
 [ 1.4419974e-01  1.5393433e-01  1.0000001e+00  8.2802188e-01
   8.7306745e-02  6.2302578e-02  1.0194063e-01  1.7433237e-02
  -4.6713978e-02 -5.9806012e-02 -7.4942425e-02  4.4832777e-02
   1.9818639e-02  1.1826560e-01  1.4005813e-01]
 [ 1.1881724e-01  9.2735447e-02  8.2802188e-01  1.0000000e+00
   1.2986918e-01  3.9691798e-02  1.3073254e-01  1.6856942e-02
  -5.3042419e-02 -8.2204491e-02 -7.5398922e-02  3.7840158e-02
   6.8672374e-04  3.6277410e-02  6.2857181e-02]
 [-4.3928705e-02 -8.4229112e-02  8.7306745e-02  1.2986918e-01
  

In [19]:
idx_sort =  np.argsort(-sim, axis = 1)
print(idx_sort)


[[ 0  1  2 14  3 10 13  7  8 11 12  9  5  4  6]
 [ 1  0  2  7  3  8  9 10 13 11 14 12  6  5  4]
 [ 2  3  1  0 14 13  6  4  5 11 12  7  8  9 10]
 [ 3  2  6  4  0  1 14  5 11 13  7 12  8 10  9]
 [ 4  6  5  3  2  7  9 10  8 11 12 13  0 14  1]
 [ 5  6  4  2 14  3 13  9 10 11  7 12  8  0  1]
 [ 6  5  4  3  2 13  9 11 10 12 14  7  1  0  8]
 [ 7  8  1  0  4 13  2  3  9  6  5 11 14 10 12]
 [ 8  7  1  0  4  9 14 10 13 11  5  2  3  6 12]
 [ 9 10 14 13  4  6  5  1  8  7  0 11 12  2  3]
 [10  9  0 14  4  6 13  5  1  8 11 12  7  2  3]
 [11 12  0  2  6  3  4  1  5  7 13 10  8  9 14]
 [12 11  6  2  0  4  3  1  5  9 13 10  7 14  8]
 [13 14  2  0  9  6 10  3  5  7  1 11  8  4 12]
 [14 13  2  9  0 10  3  5  6  1  8  4  7 11 12]]


In [20]:
for  idx, row in enumerate(idx_sort) :
    name_query = names[idx]
    print('Query Name = {}'.format(name_query))
    result = [(names[i], sim[idx, i]) for i in row[1:5] ]
    print(result)


Query Name = paulina
[('paulina', np.float32(0.6209115)), ('dicaprio', np.float32(0.14419974)), ('michelle', np.float32(0.12899187)), ('dicaprio', np.float32(0.11881724))]
Query Name = paulina
[('paulina', np.float32(0.6209115)), ('dicaprio', np.float32(0.15393433)), ('pancho', np.float32(0.11954491)), ('dicaprio', np.float32(0.09273545))]
Query Name = dicaprio
[('dicaprio', np.float32(0.8280219)), ('paulina', np.float32(0.15393433)), ('paulina', np.float32(0.14419974)), ('michelle', np.float32(0.14005813))]
Query Name = dicaprio
[('dicaprio', np.float32(0.8280219)), ('barack', np.float32(0.13073254)), ('barack', np.float32(0.12986918)), ('paulina', np.float32(0.11881724))]
Query Name = barack
[('barack', np.float32(0.70025647)), ('barack', np.float32(0.69845605)), ('dicaprio', np.float32(0.12986918)), ('dicaprio', np.float32(0.087306745))]
Query Name = barack
[('barack', np.float32(0.7008226)), ('barack', np.float32(0.69845605)), ('dicaprio', np.float32(0.06230258)), ('michelle', np.f